In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

In [2]:
import pandas as pd

from mcda.electre.discordance import discordance, discordance_marginal, discordance_comprehensive
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [3]:
alternative_names = ['fiat', 'vwPolo' , 'nissan' , 'toyota' , 'suzuki', 'ford' ]
criteria_names = ['C1', 'C2', 'C3']
profile_names = ['P1', 'P2', 'P3']

In [4]:
fiat = [100, 1000, 6.0]
vwPolo = [150, 5000, 5.7]
nissan = [110, 4000, 6.8]
toyota = [170, 3000, 5.5]
suzuki = [140, 2000, 7.8]
ford = [120, 6000, 8.0]
cars = pd.DataFrame(
    [fiat, vwPolo, nissan, toyota, suzuki, ford],
    index=alternative_names,
    columns=criteria_names,
)
cars

,C1,C2,C3
fiat,100,1000,6.0
vwPolo,150,5000,5.7
nissan,110,4000,6.8
toyota,170,3000,5.5
suzuki,140,2000,7.8
ford,120,6000,8.0


In [5]:
profile1 = [113, 1200, 7.0]
profile2 = [180, 3500, 6.7]
profile3 = [199, 3400, 8.8]
profiles = pd.DataFrame(
    [profile1, profile2, profile3],
    index=profile_names,
    columns=criteria_names
) 
profiles

,C1,C2,C3
P1,113,1200,7.0
P2,180,3500,6.7
P3,199,3400,8.8


2. Create series with thresholds and specify criteria scales and weights

In [6]:
scales = pd.Series([
    QuantitativeScale(100, 500),
    QuantitativeScale(1000, 6000, PreferenceDirection.MIN),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN)
], index=criteria_names)
scales

C1      Scale [100, 500], max direction
C2    Scale [1000, 6000], min direction
C3     Scale [0.0, 15.0], min direction
dtype: object

In [7]:
weights = pd.Series([2, 3, 5], index=criteria_names)
weights

C1    2
C2    3
C3    5
dtype: int64

In [8]:
preference = pd.Series([
    Threshold(0.1, 20),
    Threshold(0.3, 500),
    Threshold(0.1, 0.7)
], index=criteria_names)
preference

C1     Threshold: alpha=0.1, beta=20
C2    Threshold: alpha=0.3, beta=500
C3    Threshold: alpha=0.1, beta=0.7
dtype: object

In [9]:
pre_veto = pd.Series([
    None,
    Threshold(0.3, 550),
    Threshold(0.15, 0.7)
], index=criteria_names)
pre_veto

C1                               None
C2     Threshold: alpha=0.3, beta=550
C3    Threshold: alpha=0.15, beta=0.7
dtype: object

In [10]:
veto = pd.Series([
    Threshold(0.2, 20),
    Threshold(0.3, 700),
    Threshold(0.15, 0.9)
], index=criteria_names)
veto

C1      Threshold: alpha=0.2, beta=20
C2     Threshold: alpha=0.3, beta=700
C3    Threshold: alpha=0.15, beta=0.9
dtype: object

3. Let's begin with some marginal concordance calculations :)

Marginal concordance index between fiat and suzuki on the C2 criterion:

In [11]:
discordance_marginal(
    cars['C2']['fiat'],
    cars['C2']['suzuki'],
    scales['C2'],
    preference['C2'],
    veto['C2']
)

0

With pre-veto option:

In [12]:
discordance_marginal(
    cars['C2']['fiat'],
    cars['C2']['suzuki'],
    scales['C2'],
    preference['C2'],
    veto['C2'],
    pre_veto['C2']
)

0

4. Short example with comprehensive index calculations

Again, between fiat and suzuki:

In [13]:
discordance_comprehensive(
    cars.loc['fiat'],
    cars.loc['suzuki'],
    scales,
    weights,
    preference,
    veto
)

0.2

5. Concordance table between alternatives and alternatives

In [14]:
discordance(
    cars,
    scales,
    weights,
    preference,
    veto,
    pre_veto
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,0.00,0.200000,0.0,0.2,0.2,0.0
vwPolo,0.30,0.000000,0.0,0.0,0.3,0.0
nissan,0.30,0.163636,0.0,0.2,0.3,0.0
toyota,0.30,0.000000,0.0,0.0,0.0,0.0
suzuki,0.00,0.500000,0.0,0.5,0.0,0.0
ford,0.55,0.500000,0.0,1.0,0.3,0.0


Without pre-veto thresholds:

In [15]:
discordance(
    cars,
    scales,
    weights,
    preference,
    veto,
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,0.000000,0.200000,0.0,0.2,0.2,0.0
vwPolo,0.300000,0.000000,0.0,0.0,0.3,0.0
nissan,0.300000,0.163636,0.0,0.2,0.3,0.0
toyota,0.300000,0.000000,0.0,0.0,0.0,0.0
suzuki,0.271186,0.500000,0.0,0.5,0.0,0.0
ford,0.716667,0.500000,0.0,1.0,0.3,0.0


6. Concordance table between alternatives and profiles

In [16]:
alt_prof, prof_alt = discordance(
    cars,
    scales,
    weights,
    preference,
    veto,
    pre_veto,
    profiles,
)

In [17]:
alt_prof

,P1,P2,P3
fiat,0.0,0.200000,0.200000
vwPolo,0.3,0.000000,0.186667
nissan,0.3,0.200000,0.200000
toyota,0.3,0.000000,0.000000
suzuki,0.0,0.085714,0.200000
ford,0.3,0.500000,0.500000


In [18]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,0.0,0.100885,0.0,0.2,0.0,0.0
P2,0.3,0.000000,0.0,0.0,0.0,0.0
P3,0.8,0.500000,0.0,0.5,0.0,0.0
